In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import pandasql as ps

In [0]:
df1 = pd.read_csv('/content/gdrive/My Drive/Data Mining /Project Activity/Raw Data/medication_fulfillment.csv')
df2 = pd.read_csv('/content/gdrive/My Drive/Data Mining /Project Activity/Raw Data/lab_results.csv')
df3 = pd.read_csv('/content/gdrive/My Drive/Data Mining /Project Activity/Raw Data//encounter_dx.csv')
df4 = pd.read_csv('/content/gdrive/My Drive/Data Mining /Project Activity/Raw Data/encounter.csv')

In [0]:
#preprocessing df1 

In [7]:
df1.head()

Provider_Org  ...          Encounter_ID
0  ExactData Medical Health System  ...  V3046573432409333612
1  ExactData Medical Health System  ...  C2819614934032019919
2  ExactData Medical Health System  ...  N6421143613871277479
3  ExactData Medical Health System  ...  L1660181935005579193
4  ExactData Medical Health System  ...  U1554150386761469892

[5 rows x 17 columns]

In [8]:
df1.dtypes

Provider_Org      object
Order_ID          object
Member_ID         object
Prescription      object
Pharmacist_ID      int64
Pharmacy_Name     object
Dispense_Date     object
Drug_Name         object
Drug_NDC          object
Sig               object
Dispense_Qty      object
Fill_No            int64
Fill_Status       object
Dose              object
Units             object
Days_Of_Supply     int64
Encounter_ID      object
dtype: object

In [0]:
to_drop = ['Pharmacy_Name', 'Pharmacist_ID']
df1 = df1.drop(to_drop, axis = 1)

In [10]:
df1['Drug_Name'].unique()

array(['Hydrochlorothiazide', 'OMS 50', 'Prednisone', 'protamine sulfate',
       'Potassium Chl', 'nifedipine', 'Isotonic Saline (0.9%)',
       'Ciprofloxacin', 'Lisinopril', 'glyburide', 'glimepiride',
       'Insulin Nph', 'enalapril', 'insulin aspart', 'mannitol', 'AVAPRO',
       'Cefadroxil', 'Metoprolol', 'Insulin,Ultralente', 'valsartan',
       'dilaudid', 'ancef', 'oxycontin 10', 'vicodin HP 10-300',
       'Ibuprofen', 'Aspirin', 'oxycodone-acetaminophen 10-325',
       'acetaminophen', 'Nitroglycerin', 'salmeterol', 'nitroglycerin',
       'Propranolol', 'Heparin', 'metoprolol', 'Alteplase', 'Gentamicin',
       'Levofloxacin', 'Theophylline', 'ACTIVASE', 'Labetalol',
       'Furosemide', 'Morphine', 'Allopurinol', 'Captopril', 'HUMALOG',
       'Insulin Lente', 'Irbesartan', 'Insulin Reg', 'Tobramycin',
       'LANTUS', 'trimethoprim', 'BACTRIM DS', 'ATNATIV', 'LEVAQUIN'],
      dtype=object)

In [11]:
df1['Member_ID'].unique().shape

(104,)

In [12]:
df1['Order_ID'].unique().shape

(1176,)

In [13]:
null_columns=df1.columns[df1.isnull().any()]
df1[null_columns].isnull().sum()

Series([], dtype: float64)

In [0]:
df1['Dose'] = df1['Dose'] + df1['Units']

In [0]:
df1['Year'] = pd.DatetimeIndex(df1['Dispense_Date']).year
df1['Month'] = pd.DatetimeIndex(df1['Dispense_Date']).month
df1['Day'] = pd.DatetimeIndex(df1['Dispense_Date']).day

In [0]:
df1['Dispense_Date'] = pd.to_datetime(df1[['Year', 'Month', 'Day']])

In [0]:
drop = ['Year','Month','Day', 'Fill_Status', 'Fill_No', 'Units']
df1 = df1.drop(drop, axis = 1)

In [0]:
#preprocessing df2

In [19]:
df2.head()

Provider_Org  ...          Encounter_ID
0  ExactData Medical Health System  ...  X3228783097331248888
1  ExactData Medical Health System  ...  M3204724885018391695
2  ExactData Medical Health System  ...  M3204724885018391695
3  ExactData Medical Health System  ...  G7012345417035113309
4  ExactData Medical Health System  ...  G7012345417035113309

[5 rows x 21 columns]

In [20]:
df2.dtypes

Provider_Org           object
Member_ID              object
Date_Collected         object
Test_ID                object
Specialty             float64
Panel                    bool
Test_LOINC             object
Test_Name              object
Date_Resulted          object
Specimen              float64
Result_LOINC           object
Result_Name            object
Result_Status          object
Result_Description     object
Numeric_Result         object
Units                  object
Abnormal_Value        float64
Reference_Range       float64
Order_ID               object
Provider_ID             int64
Encounter_ID           object
dtype: object

In [21]:
df2.count()

Provider_Org          7509
Member_ID             7509
Date_Collected        7509
Test_ID               7509
Specialty                0
Panel                 7509
Test_LOINC            7509
Test_Name             7509
Date_Resulted         7509
Specimen                 0
Result_LOINC          7509
Result_Name           7509
Result_Status         7509
Result_Description    7509
Numeric_Result        2890
Units                 2794
Abnormal_Value           0
Reference_Range          0
Order_ID              6010
Provider_ID           7509
Encounter_ID          7509
dtype: int64

In [0]:
# Dropping columns which doesn't have data.
df2 = df2.dropna(axis = 'columns', how = 'all')

In [0]:
#Dropping Result Status column
df2 = df2.drop(['Result_Status'], axis = 1)

In [0]:
df2['Result'] = df2['Numeric_Result'] + df2['Units'] 

In [0]:
df2 = df2.drop(['Panel', 'Test_LOINC', 'Date_Resulted', 'Numeric_Result', 'Units'], axis = 1)

In [26]:
df2.head()

Provider_Org  ...    Result
0  ExactData Medical Health System  ...  250mg/dL
1  ExactData Medical Health System  ...       NaN
2  ExactData Medical Health System  ...       NaN
3  ExactData Medical Health System  ...  180mg/dL
4  ExactData Medical Health System  ...  120mg/dL

[5 rows x 12 columns]

In [0]:
#preprocessing df3

In [28]:
df3.head()

Provider_Org   code  ...  severity          Encounter_ID
0  ExactData Medical Health System  401.9  ...      mild  L8694502508698944760
1  ExactData Medical Health System  V70.0  ...       NaN  M3204724885018391695
2  ExactData Medical Health System    496  ...  critical  C2819614934032019919
3  ExactData Medical Health System  401.9  ...      mild  S4571692645992867370
4  ExactData Medical Health System  V70.0  ...       NaN  J6262874375300991101

[5 rows x 6 columns]

In [0]:
df3.drop(df3.columns[[0, 1, 2]], axis = 1, inplace = True)

In [30]:
df3.head()

description  severity          Encounter_ID
0                           Hypertension      mild  L8694502508698944760
1            General medical examination       NaN  M3204724885018391695
2  Chronic Obstructive Pulmonary Disease  critical  C2819614934032019919
3                           Hypertension      mild  S4571692645992867370
4            General medical examination       NaN  J6262874375300991101

In [0]:
#preprocessing df4

In [32]:
df4.head()

Provider_Org  ...                 Disposition
0  ExactData Medical Health System  ...  ReleasedWithoutLimitations
1  ExactData Medical Health System  ...  ReleasedWithoutLimitations
2  ExactData Medical Health System  ...  ReleasedWithoutLimitations
3  ExactData Medical Health System  ...  ReleasedWithoutLimitations
4  ExactData Medical Health System  ...  ReleasedWithoutLimitations

[5 rows x 28 columns]

In [33]:
df4.count()

Provider_Org                    5447
Encounter_ID                    5447
Member_ID                       5447
Provider_ID                     5447
Provider_NPI                    5447
Clinic_ID                       5447
Encounter_DateTime              5447
Encounter_Description           5447
CC                              2237
Episode_ID                      5447
Patient_DOB                     5447
Patient_Gender                  5447
Facility_Name                      0
Provider_Name                   5447
Specialty                       3543
Clinic_Type                     5447
lab_orders_count                5447
lab_results_count               5447
medication_orders_count         5447
medication_fulfillment_count    5447
vital_sign_count                5447
therapy_orders_count            5447
therapy_actions_count           5447
immunization_count              5447
Has_Appt                        5447
SOAP_Note                       3352
consult_ordered                  554
D

In [0]:
df4 = df4.dropna(axis = 'columns', how = 'all')

In [0]:
to_drop = ['Provider_Name', 'Clinic_ID','Has_Appt']
df4 = df4.drop(to_drop, axis=1)

In [0]:
df4.rename(columns = {'Patient_DOB':'Age'}, inplace = True) 
df4['Age'] = pd.to_datetime(df4['Age'], format='%m/%d/%Y')
t_date = '2006-01-01 00:00:0.000000'
df4["Age"] = (pd.to_datetime(t_date) - df4['Age']).astype('<m8[Y]') 

In [0]:
year = pd.DatetimeIndex(df4['Encounter_DateTime']).year
month = pd.DatetimeIndex(df4['Encounter_DateTime']).month
day = pd.DatetimeIndex(df4['Encounter_DateTime']).day

In [0]:
def _month(x):
    if x == 1:
        return 'Jan'
    if x == 2:
        return 'Feb'
    if x == 3:
        return 'Mar'
    if x == 4:
        return 'Apr'
    if x == 5:
        return 'May'
    if x == 6:
        return 'Jun'
    if x == 7:
        return 'Jul'
    if x == 8:
        return 'Aug'
    if x == 9:
        return 'Sep'
    if x == 10:
        return 'Oct'
    if x == 11:
        return 'Nov'
    if x == 12:
        return 'Dec'

In [0]:
year = np.asarray(year)
yea = []
for i in range (0, len(year)):
    s = str(year[i])
    yea.append(s)
res = []
for i in range(0, len(month)):
    r = _month(month[i])
    res.append(r)
date = []
for i in range(0, len(day)):
    l = str(day[i])
    date.append(l)

ans1 = np.char.add(date, ' ')
ans2 = np.char.add(ans1, res)
ans3 = np.char.add(ans2, ' ')
ans = np.char.add(ans3, yea)
col = pd.DataFrame(data=ans)
df4["Encounter_DateTime"] = col
df4.rename(columns = {'Encounter_DateTime':'Encounter_Date'}, inplace = True) 

Merging and Analysis

In [40]:
df1.columns

Index(['Provider_Org', 'Order_ID', 'Member_ID', 'Prescription',
       'Dispense_Date', 'Drug_Name', 'Drug_NDC', 'Sig', 'Dispense_Qty', 'Dose',
       'Days_Of_Supply', 'Encounter_ID'],
      dtype='object')

In [41]:
df2.columns

Index(['Provider_Org', 'Member_ID', 'Date_Collected', 'Test_ID', 'Test_Name',
       'Result_LOINC', 'Result_Name', 'Result_Description', 'Order_ID',
       'Provider_ID', 'Encounter_ID', 'Result'],
      dtype='object')

In [42]:
df3.columns

Index(['description', 'severity', 'Encounter_ID'], dtype='object')

In [43]:
df4.columns

Index(['Provider_Org', 'Encounter_ID', 'Member_ID', 'Provider_ID',
       'Provider_NPI', 'Encounter_Date', 'Encounter_Description', 'CC',
       'Episode_ID', 'Age', 'Patient_Gender', 'Specialty', 'Clinic_Type',
       'lab_orders_count', 'lab_results_count', 'medication_orders_count',
       'medication_fulfillment_count', 'vital_sign_count',
       'therapy_orders_count', 'therapy_actions_count', 'immunization_count',
       'SOAP_Note', 'consult_ordered', 'Disposition'],
      dtype='object')

In [44]:
df2['Provider_ID'].unique().shape

(99,)

In [45]:
df3['Encounter_ID'].unique().shape

(2717,)

In [46]:
df4['Provider_ID'].unique().shape

(155,)

In [47]:
df3['Encounter_ID'].shape

(3063,)

In [48]:
df2['Member_ID'].unique().shape

(104,)

In [49]:
df3.shape

(3063, 3)

In [50]:
df2.shape

(7509, 12)

In [51]:
df2.head()

Provider_Org  ...    Result
0  ExactData Medical Health System  ...  250mg/dL
1  ExactData Medical Health System  ...       NaN
2  ExactData Medical Health System  ...       NaN
3  ExactData Medical Health System  ...  180mg/dL
4  ExactData Medical Health System  ...  120mg/dL

[5 rows x 12 columns]

In [52]:
df2.isnull().sum()

Provider_Org             0
Member_ID                0
Date_Collected           0
Test_ID                  0
Test_Name                0
Result_LOINC             0
Result_Name              0
Result_Description       0
Order_ID              1499
Provider_ID              0
Encounter_ID             0
Result                4715
dtype: int64

In [53]:
df1.head()

Provider_Org  ...          Encounter_ID
0  ExactData Medical Health System  ...  V3046573432409333612
1  ExactData Medical Health System  ...  C2819614934032019919
2  ExactData Medical Health System  ...  N6421143613871277479
3  ExactData Medical Health System  ...  L1660181935005579193
4  ExactData Medical Health System  ...  U1554150386761469892

[5 rows x 12 columns]

In [54]:
df3.head()

description  severity          Encounter_ID
0                           Hypertension      mild  L8694502508698944760
1            General medical examination       NaN  M3204724885018391695
2  Chronic Obstructive Pulmonary Disease  critical  C2819614934032019919
3                           Hypertension      mild  S4571692645992867370
4            General medical examination       NaN  J6262874375300991101

In [55]:
df3.isnull().sum()

description       0
severity        696
Encounter_ID      0
dtype: int64

In [0]:
#merging the df1 and df3 tables
query = ps.sqldf("""select df1.*,
df3.description,
df3.severity 
from df1 
left join df3
on df1.Encounter_ID=df3.Encounter_ID""")

In [57]:
query

Provider_Org  ...  severity
0     ExactData Medical Health System  ...      None
1     ExactData Medical Health System  ...  critical
2     ExactData Medical Health System  ...      None
3     ExactData Medical Health System  ...      None
4     ExactData Medical Health System  ...      None
...                               ...  ...       ...
1171  ExactData Medical Health System  ...      None
1172  ExactData Medical Health System  ...      None
1173  ExactData Medical Health System  ...      None
1174  ExactData Medical Health System  ...    severe
1175  ExactData Medical Health System  ...      None

[1176 rows x 14 columns]

In [58]:
query1 = ps.sqldf("""select order_id
from df2 
group by order_id
having count(*)>2""")
query1

Order_ID
0            None
1    050103-00172
2    050103-74209
3    050112-15188
4    050118-10481
..            ...
433  141104-75215
434  141117-38908
435  141124-40931
436  141126-03935
437  141215-64971

[438 rows x 1 columns]

In [59]:
df2.head()

Provider_Org  ...    Result
0  ExactData Medical Health System  ...  250mg/dL
1  ExactData Medical Health System  ...       NaN
2  ExactData Medical Health System  ...       NaN
3  ExactData Medical Health System  ...  180mg/dL
4  ExactData Medical Health System  ...  120mg/dL

[5 rows x 12 columns]

In [60]:
query2 = ps.sqldf("""select * from df2 
where order_id='050103-00172'""")
query2


Provider_Org  ...    Result
0  ExactData Medical Health System  ...  180mg/dL
1  ExactData Medical Health System  ...  120mg/dL
2  ExactData Medical Health System  ...   45mg/dL
3  ExactData Medical Health System  ...  130mg/dL

[4 rows x 12 columns]

In [61]:
query3 = ps.sqldf("""select order_id, Result_LOINC,count(*) as cnt from df2 group by order_id, Result_LOINC order by cnt desc""")
query3

Order_ID Result_LOINC  cnt
0             None      28631-0  535
1             None      32422-8  235
2             None      11402-5  220
3             None      28615-3  154
4             None      10204-6  124
...            ...          ...  ...
6013  141218-89931      10206-1    1
6014  141219-12162      24693-4    1
6015  141219-19595       2093-3    1
6016  141219-31157       2345-7    1
6017  141221-58441      17856-6    1

[6018 rows x 3 columns]

In [0]:
data = ps.sqldf("""select query.*,
df2.Test_Name,
df2.Result_LOINC,
df2.Result_Name
from query 
left join df2
on query.Encounter_ID=df2.Encounter_ID""")

In [63]:
data

Provider_Org      Order_ID  ... Result_LOINC Result_Name
0     ExactData Medical Health System  050630-23852  ...         None        None
1     ExactData Medical Health System  050312-01255  ...       2019-8       PaCO2
2     ExactData Medical Health System  050312-01255  ...       2703-7        PaO2
3     ExactData Medical Health System  050312-01255  ...      19945-5        FEV1
4     ExactData Medical Health System  050312-01255  ...      19926-5    FEV1/FVC
...                               ...           ...  ...          ...         ...
1600  ExactData Medical Health System  141219-31653  ...         None        None
1601  ExactData Medical Health System  140303-38297  ...         None        None
1602  ExactData Medical Health System  140318-98860  ...         None        None
1603  ExactData Medical Health System  140303-90178  ...      17856-6       HbA1c
1604  ExactData Medical Health System  141124-53578  ...         None        None

[1605 rows x 17 columns]

In [64]:
df4.head()

Provider_Org  ...                 Disposition
0  ExactData Medical Health System  ...  ReleasedWithoutLimitations
1  ExactData Medical Health System  ...  ReleasedWithoutLimitations
2  ExactData Medical Health System  ...  ReleasedWithoutLimitations
3  ExactData Medical Health System  ...  ReleasedWithoutLimitations
4  ExactData Medical Health System  ...  ReleasedWithoutLimitations

[5 rows x 24 columns]

In [65]:
df4['vital_sign_count'].unique()

array([7, 0])

In [0]:
merged_data = ps.sqldf("""select data.*,
df4.CC 
from data 
left join df4
on data.Encounter_ID=df4.Encounter_ID""")

In [67]:
merged_data

Provider_Org  ...                          CC
0     ExactData Medical Health System  ...                        None
1     ExactData Medical Health System  ...            critical dyspnea
2     ExactData Medical Health System  ...            critical dyspnea
3     ExactData Medical Health System  ...            critical dyspnea
4     ExactData Medical Health System  ...            critical dyspnea
...                               ...  ...                         ...
1600  ExactData Medical Health System  ...                        None
1601  ExactData Medical Health System  ...                        None
1602  ExactData Medical Health System  ...                        None
1603  ExactData Medical Health System  ...  moderate urinary frequency
1604  ExactData Medical Health System  ...                        None

[1605 rows x 18 columns]

In [68]:
merged_data['description'].unique()

array([None, 'Chronic Obstructive Pulmonary Disease', 'Type 2 Diabetes',
       'Type 1 Diabetes', 'Hemorrhagic Stroke', 'Acute Renal Failure',
       'critical distress', 'critical hysteria', 'Myocardial Infarction',
       'Hypertension', 'Embolic Stroke', 'Chronic Renal Failure',
       'Pyelonephritis', 'critical agony',
       'Chronic Congestive Heart Failure'], dtype=object)

In [69]:
drugname = ps.sqldf("""select Drug_Name, description,severity, CC, count(*) as cnt from merged_data group by Drug_Name, description order by cnt desc""")
drugname

Drug_Name  ...  cnt
0                           OMS 50  ...  311
1                    Potassium Chl  ...  147
2                    Ciprofloxacin  ...  105
3           Isotonic Saline (0.9%)  ...  101
4                       Lisinopril  ...   83
..                             ...  ...  ...
76                      metoprolol  ...    1
77  oxycodone-acetaminophen 10-325  ...    1
78                    trimethoprim  ...    1
79                    trimethoprim  ...    1
80                       valsartan  ...    1

[81 rows x 5 columns]

In [0]:
merged_data.to_csv("merged_data.csv")